# Quantum simulation of the 2D Ising Model

In this Python + Q# notebook we demonstrate resource estimation for quantum dynamics,
specifically the simulation of an Ising model Hamiltonian on an $N \times N$ 2D
lattice using a *fourth-order Trotter Suzuki product formula* assuming a 2D
qubit architecture with nearest-neighbor connectivity.

First, we connect to the Azure quantum service and load the necessary packages.

In [ ]:
from azure.quantum import Workspace
from azure.quantum.target.microsoft import MicrosoftEstimator, QubitParams, QECScheme
import qsharp

In [ ]:
workspace = Workspace(
            resource_id = "",
            location = "")



In [ ]:
qsharp.packages.add("Microsoft.Quantum.Numerics")


## Background: 2D Ising model

The Ising model is a mathematical model of ferromagnetism in a lattice (in our case a 2D square lattice) with two kinds of terms in the Hamiltonian: (i) an interaction term between adjacent sites and (ii) an external magnetic field acting at each site. For our purposes, we consider a simplified version of the model where the interaction terms have the same strength and the external field strength is the same at each site.
Formally, the Ising model Hamiltonian on an $N \times N$ lattice we consider is formulated as:

$$
H = \underbrace{-J \sum_{i, j} Z_i Z_j}_{B} + \underbrace{g \sum_j X_j}_{A}
$$
where $J$ is the interaction strength, $g$ is external field strength.

The time evolution $e^{-iHt}$ for the Hamiltonian is simulated with the fourth-order product formula so that any errors in simulation are sufficiently small. Essentially, this is done by simulating the evolution for small slices of time $\Delta$ and repeating this for `nSteps` $= t/\Delta$ to obtain the full time evolution. The Trotter-Suzuki formula for higher orders can be recursively defined using a *fractal decomposition* as discussed in Section 3 of [[Hatanao and Suziki's survey](https://link.springer.com/chapter/10.1007/11526216_2)]. Then the fourth order formula $U_4(\Delta)$ can be constructed using the second-order one $U_2(\Delta)$ as follows.
$$
\begin{aligned}
U_2(\Delta) & = e^{-iA\Delta/2} e^{-iB\Delta} e^{-iA\Delta/2}; \\
U_4(\Delta) & = U_2(p\Delta)U_2(p\Delta)U_2((1 - 4p)\Delta)U_2(p\Delta)U_2(p\Delta); \\
p & = (4 - 4^{1/3})^{-1}.
\end{aligned}
$$

For the rest of the notebook, we will present the code that computes the time evolution in a step by step fashion.

## Implementation

### Helper functions

We will allocate all qubits in the 2D lattice in a two-dimensional array.

Note that expanding $U_4(\Delta)$ to express it in terms of $A, B$ gives:
$$
U_4(\Delta) = e^{-iAp\Delta/2} e^{-iBp\Delta} e^{-iAp\Delta} e^{-iBp\Delta} e^{-iA(1 - 3p)\Delta/2} e^{-iB(1-4p)\Delta} e^{-iA(1 - 3p)\Delta/2} e^{-iBp\Delta} e^{-iAp\Delta} e^{-iBp\Delta} e^{-iAp\Delta/2}
$$

The above equation with $11$ exponential terms works for one time step. For `nSteps` $> 1$ time steps, some adjacent terms can be merged to give $10t+1$ exponential terms for $e^{-iHt}$.

The function below creates a sequence containing the constant factors that will be applied with $A$ and $B$, respectively, in the exponential sequence of the above formula.

In [ ]:
%%qsharp
function SetAngleSequence(p : Double, dt : Double, J : Double, g : Double) : Double[] {

    let len1 = 3;
    let len2 = 3;
    let valLength = 2*len1+len2+1;
    mutable values = [0.0, size=valLength];

    let val1 = J*p*dt;
    let val2 = -g*p*dt;
    let val3 = J*(1.0 - 3.0*p)*dt/2.0;
    let val4 = g*(1.0 - 4.0*p)*dt/2.0;

    for i in 0..len1 {
        
        if (i % 2 == 0) {
            set values w/= i <- val1;
        }
        else {
            set values w/= i <- val2;
        }

    }

    for i in len1+1..len1+len2 {
        if (i % 2 == 0) {
            set values w/= i <- val3;
        }
        else {
            set values w/= i <- val4;
        }
    }

    for i in len1+len2+1..valLength-1 {
        if (i % 2 == 0) {
            set values w/= i <- val1;
        }
        else {
            set values w/= i <- val2;
        }
    }
    return values;
}

### Quantum operations

There are two kinds of Pauli exponentials needed for simulating the time evolution of an Ising Model:
- The transverse field $e^{-iX\theta}$ applied to each qubit for an angle $\theta$;
- $e^{-i (Z \otimes Z)\theta}$ applied to neighboring pairs of qubits in the lattice.

The operation below applies $e^{-iX\theta}$ on all qubits in the 2D lattice.

In [ ]:
%%qsharp
operation ApplyAllX(n : Int, qArr : Qubit[][], theta : Double) : Unit {
    // This applies `Rx` with an angle of `2.0 * theta` to all qubits in `qs`
    // using partial application
    for row in 0..n-1 {
        ApplyToEach(Rx(2.0 * theta, _), qArr[row]);
    }
}

The next operation below applies $e^{-i(Z \otimes Z)\theta}$ on overlapping pairs of neighboring qubits. We decompose this term into a single qubit $e^{-iZ\theta}$ term (implemented as an `Rz` rotation) conjugated by `CNOT`s to entangle the neighboring qubits following Section 4.2 of [[Whitfield et al.](https://www.tandfonline.com/doi/abs/10.1080/00268976.2011.552441)].

Observe that unlike the previous case, it is not possible to simultaneously apply all the rotations in one go. For example, while applying the rotation on qubits at $(0, 0)$ and $(0, 1)$, it is not possible to also apply the rotation on qubits $(0, 1)$ and $(0, 2)$. Instead, we try to apply as many rotations as possible. This is broken up as follows:
- in the vertical (resp. horizontal) direction of the 2D lattice as chosen by `dir`,
- consider pairs starting with an even (resp. odd) index as given by `grp`;
- apply the exponential to all such pairs in the lattice.

In [ ]:
%%qsharp
operation ApplyDoubleZ(n : Int, m : Int, qArr : Qubit[][], theta : Double, dir : Bool, grp : Bool) : Unit {
    let start = grp ? 1 | 0;    // Choose either odd or even indices based on group number
    let P_op = [PauliZ, PauliZ];
    let c_end = dir ? m-1 | m-2;
    let r_end = dir ? m-2 | m-1;

    for row in 0..r_end {
        for col in start..2..c_end {    // Iterate through even or odd columns based on `grp`

            let row2 = dir ? row+1 | row;
            let col2 = dir ? col | col+1;

            Exp(P_op, theta, [qArr[row][col], qArr[row2][col2]]);
        }
    }
}


The next operation puts everything together and calls the operations needed to
simulate the Ising model Hamiltonian using a fourth order product formula.
Observe that the `ApplyDoubleZ` operation is called four times for different
choices of direction and starting index to ensure all possible pairs of qubits
are appropriately considered.

The various parameters taken in by the operation correspond to:

- `J`, `g`: parameters by which the Hamiltonian terms are scaled.
- `N`: size of the square lattice.
- `totTime`: the number of Trotter steps.
- `dt` : the step size for the simulation, sometimes denoted as $\Delta$.
- `eps`: the precision for arbitrary rotations.

In [ ]:
%%qsharp
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;

operation IsingModel2DSim(N1 : Int, N2 : Int, J : Double, g : Double, totTime : Double, dt : Double) : Unit {
    use qs = Qubit[N1*N2];
    let qubitArray = Chunks(N2, qs);

    let p = 1.0 / (4.0 - PowD(4.0, 1.0 / 3.0));
    let t = Ceiling(totTime / dt);

    let seqLen = 10 * t + 1;

    let angSeq = SetAngleSequence(p, dt, J, g);

    for i in 0..seqLen - 1 {
        let theta = (i==0 or i==seqLen-1) ? J*p*dt/2.0 | angSeq[i%10];

        // for even indexes
        if i % 2 == 0 {
            ApplyAllX(N1, qubitArray, theta);
        } else {
            // iterate through all possible combinations for `dir` and `grp`.
            for (dir, grp) in [(true, true), (true, false), (false, true), (false, false)] {
                ApplyDoubleZ(N1, N2, qubitArray, theta, dir, grp);
            }
        }
    }
}


## Running the experiment

Next, we are estimating the physical resource estimates to simulate the Ising
model Hamiltonian for a $10 \times 10$ lattice with $J = g = 1.0$, total time
$10$, step size $0.9$. As configurations for the
experiment we use all six pre-defined qubit parameters.  As pre-defined QEC
scheme we are using `surface_code` with gate-based qubit parameters (default),
and `floquet_code` with Majorana based qubit parameters.

In [ ]:
estimator = MicrosoftEstimator(workspace)

labels = ["Gate-based μs, 10⁻3", "Gate-based μs, 10⁻⁴", "Gate-based ns, 10⁻3", "Gate-based ns, 10⁻⁴", "Majorana ns, 10⁻⁴", "Majorana ns, 10⁻6"]

params = estimator.make_params(num_items=6)
params.arguments["N1"] = 10
params.arguments["N2"] = 10
params.arguments["J"] = 1.0
params.arguments["g"] = 1.0
params.arguments["totTime"] = 10.0
params.arguments["dt"] = 0.9
params.items[0].qubit_params.name = QubitParams.MAJ_NS_E4
params.items[0].qec_scheme.name = QECScheme.FLOQUET_CODE
params.items[1].qubit_params.name = QubitParams.MAJ_NS_E4
params.items[1].qec_scheme.name = QECScheme.FLOQUET_CODE
params.items[2].qubit_params.name = QubitParams.MAJ_NS_E4
params.items[2].qec_scheme.name = QECScheme.FLOQUET_CODE
params.items[3].qubit_params.name = QubitParams.MAJ_NS_E4
params.items[3].qec_scheme.name = QECScheme.FLOQUET_CODE
params.items[4].qubit_params.name = QubitParams.MAJ_NS_E4
params.items[4].qec_scheme.name = QECScheme.FLOQUET_CODE
params.items[5].qubit_params.name = QubitParams.MAJ_NS_E4
params.items[5].qec_scheme.name = QECScheme.FLOQUET_CODE


We are submitting a resource estimation job with all target parameter configurations.

In [ ]:
job = estimator.submit(IsingModel2DSim, input_params=params)
results = job.get_results()


## Visualizing and understanding the results

### Result summary table

In [ ]:
results.summary_data_frame(labels=labels)


### Space chart


The distribution of physical qubits used for the execution of the algorithm instructions and the supporting T factories can provide us valuable information to guide us in applying space and time optimizations. We can visualize this distribution for each set of qubit parameters to understand the differences in physical qubit distribution for each configuration.

To show the space chart for a configuration from our experiment, use the following syntax:

```
    # Use the index of the desired configuration you want to visualize to get the results for that configuration.
    
    results[<item index>].diagram.space

    # For example, this command will produce a chart showing the distribution of physical qubits for the Gate-based µs, 10⁻³ configuration set.
    results[0].diagram.space 
```

Below, let's visualize the space diagrams for one of the gate-based and one of the Majorana configurations.

> You must run the `results[<item index>].diagram.space` command in a separate code cell per label.
>
> You cannot visualize the time and space diagrams in the same cell.
>
> If you run an algorithm which only has one configured set of qubit parameters and one result set, specifying the label would not be necessary. You could simply run `result.diagram.time`

In [ ]:
# "Gate-based ns, 10⁻³"
results[2].diagram.space


In [ ]:
# "Majorana ns, 10⁻⁴"
results[4].diagram.space


### Time chart
We can also visualize the time required to execute the algorithm as it relates to each T factory invocation runtime and the number of T factory invocations.


To show the time chart for a configuration from our experiment, use the following syntax:

```
    # Use the index of the desired configuration you want to visualize to get the results for that configuration.
    
    results[<item index>].diagram.time

    # For example, this command will produce a chart showing the distribution of physical qubits for the Gate-based µs, 10⁻³ configuration set.
    results[0].diagram.time
```

Below, let's visualize the space diagrams for one each of the gate-based and Majorana configurations.

> You must run the `results[<item index>].diagram.time` command in a separate code cell per label.
>
> You cannot visualize the time and space diagrams in the same cell.
>
> If you run an algorithm which only has one configured set of qubit parameters and one result set, specifying the label would not be necessary. You could simply run `result.diagram.time`

In [ ]:
results[2].diagram.time

In [ ]:
results[4].diagram.time

From the results we can observe that a large fraction of physical qubits is used
for the T factories.  To understand why, it's important to remark that the
overall algorithm runtime is determined based on the number of logical
operations (also called logical cycles or logical depth).  The runtime limits
the number of invocations of a single T factory.  The total number of T factory
copies is computed based on the total number of required T states divided by the
number of possible invocations.  Therefore, if the algorithm would run longer, a
T factory can be invoked more often, which may allow to compute all required T
states with less T factory copies.

Since T factory fraction is high, while at the same time the physical runtime is
relatively small, this is a good opportunity for a space-time optimization based
on the logical depth.  We can make an algorithm run longer, by inserting no-op
(no operation or idle) operations.  We do this using the `logical_depth_factor`
constraint. For example, a value of 2 means that the number of cycles should be
twice as much, i.e., one no-op per operation; or, a value of 1.5 means that the
number of cycles is 50% more, i.e., one no-op for every two operations.

Please note that the algorithm runtime may increase by a larger factor than the
`logical_depth_factor`.  This is because no-ops also can incur logical errors,
and therefore the required logical error rate is lower, which in turn may
increase the required code distance, therefore leading a to a longer execution
time of a logical cycle.

In the balanced implementation that is described in the paper, we increase the
logical depth by a factor of 10.  We do this by updating the `params` variable.
All other parameters remain unchanged.

In [ ]:
params.constraints.logical_depth_factor = 10

job = estimator.submit(IsingModel2DSim, input_params=params)
results_balanced = job.get_results()


We print the results for the balanced implementation as a summary.

In [ ]:
results_balanced.summary_data_frame(labels=labels)


Note how the T factory fraction is much smaller now and the number of total
physical qubits decreased as a result.  The logical depth increased exactly by a
factor of 10, whereas the runtime increased by a factor of at least 10, since in
most cases the code distance is higher.

To better understand how the balanced implementation changed the space distribution and runtime of the algorithm, try visualizing the new result set using the space and time diagrams discussed above.

In [ ]:
# Feel free to try out different configuration sets by changing the index. 
# This example produces the space diagram for the Gate-based ns, 10⁻³ configuration set.
results_balanced[2].diagram.space


In [ ]:
# Feel free to try out different configuration sets by changing the index. 
# This example produces the time diagram for the Gate-based ns, 10⁻³ configuration set.
results_balanced[2].diagram.time


You could even compare the previous implementation with the balanced implementation by re-running the space and time diagrams with the original result set and comparing to the new diagrams.

In [ ]:
# Feel free to try out different configuration sets by changing the index. 
# This example produces the space diagram for the Gate-based ns, 10⁻³ configuration set.
results[2].diagram.space


In [ ]:
# Feel free to try out different configuration sets by changing the index. 
# This example produces the time diagram for the Gate-based ns, 10⁻³ configuration set.
results[2].diagram.time


### Limiting T factories

Another way to balance the number of physical qubits allocates to the algorithm and T-factories without necessarily increasing the number of algorithmic qubits is to enforce a limit on the maximum number of T factories that can be used by the algorithm with `contraints.max_t_factories`.

In [ ]:
params.constraints.max_t_factories = 40
params.constraints.logical_depth_factor = 1

job = estimator.submit(IsingModel2DSim, input_params=params)
results_limited = job.get_results()

In [ ]:
results_limited.summary_data_frame(labels=labels)

The time and space diagram for one of the configurations can be seen below.

In [ ]:
# Feel free to try out different configuration sets by changing the index. 
# This example produces the space diagram for the Gate-based ns, 10⁻³ configuration set.
results_limited[2].diagram.space

In [ ]:
# Feel free to try out different configuration sets by changing the index. 
# This example produces the time diagram for the Gate-based ns, 10⁻³ configuration set.
results_limited[2].diagram.time

## Next steps

Feel free to use this table as a starting point for your own experiments.  For
example, you can

* explore how the results change by modifying the operation arguments of the Ising
  model instance
* explore space- and time-trade-offs by changing the value for
  `logical_depth_factor` or `max_t_factories`
* Visualize these trade-offs with the space and time diagrams
* use other or customized qubit parameters